
# Tag counting

In [1]:
DATA_PATH = 'gs://yn-ml-models/'
!gsutil ls -l $DATA_PATH

    294811  2018-10-09T06:57:00Z  gs://yn-ml-models/VOE-Image-1.jpg
 114441858  2018-10-03T03:43:53Z  gs://yn-ml-models/basic-nlp-models.zip
 172272626  2019-03-15T08:49:31Z  gs://yn-ml-models/chatbot_acb_v1.tgz
 172124430  2019-03-15T08:49:41Z  gs://yn-ml-models/chatbot_acb_v2.tgz
 172138318  2019-03-19T07:42:42Z  gs://yn-ml-models/chatbot_acb_v3.tgz
 171482026  2019-03-22T11:06:00Z  gs://yn-ml-models/chatbot_acb_v4.tgz
 171480513  2019-03-26T08:56:57Z  gs://yn-ml-models/chatbot_acb_v5.tgz
 171242293  2019-03-27T08:13:24Z  gs://yn-ml-models/chatbot_acb_v6.tgz
 171201016  2019-03-29T07:17:17Z  gs://yn-ml-models/chatbot_acb_v7.tgz
 171200760  2019-03-29T15:43:08Z  gs://yn-ml-models/chatbot_acb_v8.tgz
  42514535  2018-11-07T09:06:19Z  gs://yn-ml-models/cnn_tokenizer.pkl
   1240328  2020-01-15T02:40:05Z  gs://yn-ml-models/customerservice_clf_model_231219.pkl
  32327752  2018-09-12T03:27:28Z  gs://yn-ml-models/draft_model_v1.0.h5
  32327752  2018-09-12T03:32:26Z  gs://yn-ml-models/draft_te

In [ ]:
"""
For getting file from gcloud
"""
# DATA_PATH = 'gs://yn-ml-models/tag_clf/temp/'
# !gsutil ls -l $DATA_PATH

# # Cloning data to the context
# !gsutil cp -r gs://yn-ml-models/tag_clf/temp/* .

         0  2020-11-02T03:27:27Z  gs://yn-ml-models/tag_clf/temp/
 984191659  2020-11-02T11:10:46Z  gs://yn-ml-models/tag_clf/temp/CLOUD_August_MILK.csv
 389164508  2020-11-02T11:21:38Z  gs://yn-ml-models/tag_clf/temp/CLOUD_July_MILK.csv
 580825570  2020-11-02T10:38:27Z  gs://yn-ml-models/tag_clf/temp/CLOUD_MARJUN_MILK.csv
2232045828  2020-11-02T12:26:30Z  gs://yn-ml-models/tag_clf/temp/CLOUD_September_MILK.csv
     49693  2020-11-02T11:21:39Z  gs://yn-ml-models/tag_clf/temp/CLOUD_milk_attribute_mapping.py
     65381  2020-11-02T11:21:40Z  gs://yn-ml-models/tag_clf/temp/CLOUD_milk_kw_ref.py
     65381  2020-11-02T07:29:20Z  gs://yn-ml-models/tag_clf/temp/milk_kw_ref.py
  76017368  2020-11-17T02:43:04Z  gs://yn-ml-models/tag_clf/temp/sentiment_mapping_August.csv
  39678861  2020-11-17T02:44:11Z  gs://yn-ml-models/tag_clf/temp/sentiment_mapping_July.csv
  79867511  2020-11-17T06:31:37Z  gs://yn-ml-models/tag_clf/temp/sentiment_mapping_MARJUN.csv
 245651885  2020-11-17T02:51:18Z  gs://yn-

In [ ]:
"""
For getting file from local
"""
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

'\nFor getting file from local\n'

In [ ]:
"""
For getting file from gdrive
"""

from google.colab import drive
drive.mount('/gdrive')

DATA_PATH = "/gdrive/My Drive/nlp/sentiment/temp"
!ls -l "$DATA_PATH"
!cp -r "$DATA_PATH" .

Mounted at /gdrive
total 4088297
-rw------- 1 root root  984191659 Nov  2 12:30 CLOUD_August_MILK.csv
-rw------- 1 root root  389164508 Nov  2 12:30 CLOUD_July_MILK.csv
-rw------- 1 root root  580825570 Nov  2 12:30 CLOUD_MARJUN_MILK.csv
-rw------- 1 root root      49693 Nov  2 12:31 CLOUD_milk_attribute_mapping.py
-rw------- 1 root root      65381 Nov  2 12:31 CLOUD_milk_kw_ref.py
-rw------- 1 root root 2232045828 Nov  2 12:31 CLOUD_September_MILK.csv
-rw------- 1 root root      65381 Nov  2 12:31 milk_kw_ref.py
-rw------- 1 root root       6313 Nov  2 12:31 tag_id_by_name.py


In [ ]:
import glob
import os 
from os import path

datafiles = []
for path in glob.iglob(os.path.join('/content/temp', "*.csv")):
    datafiles.append(path)

datafiles

['/content/temp/CLOUD_MARJUN_MILK.csv',
 '/content/temp/CLOUD_September_MILK.csv',
 '/content/temp/CLOUD_August_MILK.csv',
 '/content/temp/CLOUD_July_MILK.csv']

In [ ]:
# IMPORT LIBRARIES

import pandas as pd
from temp.CLOUD_milk_attribute_mapping import MILK_ATTRIBUTE_MAPPING
import functools as ft

# Setting pandas to display all content 
pd.set_option("display.max_colwidth", 1)

In [ ]:
"""
Map tagid to tagname
"""
tag_name_by_tagid = lambda tagid: [key for key in TAG_ID.keys() if tagid in TAG_ID[key]]

"""
Get tagid list from tag string
"""
get_tagid_list_from_string = lambda tagstr: [int(t) for t in tagstr.strip('[').strip(']').split(', ') if t!='']

In [ ]:
def count_by_tag(keys, tagseries):
    """
    Return 2 dictionaries:
        __tag_count: to map each tag to the corresponding number of document with that tag
        __sgl_tag_count: to map each tag with corresponding number of documents have that tag as single tag
    """
    __tag_count = dict((k, 0) for k in keys)
    __sgl_tag_count = dict((k, 0) for k in keys)

    for tagstring in tagseries: 
        tags = get_tagid_list_from_string(tagstring)
        
        # Nếu list tag không có tag nào thì cộng 1 vào 'no-tag'
        if tags==[]: __tag_count['no-tag'] += 1

        # Map tagid to tagname
        # Any tagid connot be mapped: ignore it
        tagnames = set()
        for tag in tags: 
            try:
                tagnames.add(MILK_ATTRIBUTE_MAPPING[tag])
            except:
                pass

        # Voi moi tag có trong tag list của document thì add 1 vào key tương ứng trong dictionary __tag_count
        # Nếu không tìm thấy tag tương ứng thì add 1 vào key undefined
        for tagname in tagnames:
            __tag_count[tagname] += 1

        # Nếu list tags chỉ có 1 tag "T" thì add 1 vào T trong dictionary 
        # Nếu không tìm thấy tag tương ứng thì add 1 vào key undefined
        if len(tagnames) == 1:
            (element,) = tagnames
            __sgl_tag_count[element] += 1

    return __tag_count, __sgl_tag_count

In [ ]:
def get_tagname_list_from_string(tagstring): 
    tags = get_tagid_list_from_string(tagstring)
    tagnames = set()
    for tag in tags: 
        try:
            tagnames.add(MILK_ATTRIBUTE_MAPPING[tag])
        except:
            pass
    return tagnames

def get_sgl_tag_doc(df):
    def mapfunc(row):
        # print(row)
        tagnames = get_tagname_list_from_string(row.tags)
        if len(tagnames) == 1:
            return (row.id, tagnames)
    df = df.apply(mapfunc, axis=1).dropna()
    return df

In [ ]:
RESULT_PATH = '/content/summary/'

if not os.path.exists(RESULT_PATH):
    os.makedirs(RESULT_PATH)

In [ ]:
__TAGSET__ = set(MILK_ATTRIBUTE_MAPPING.values())
__TAGSET__.add('no-tag')
__SINGLE_TAG__ = []

for file in datafiles:
    df = pd.read_csv(file)

    #========================Overview=========================#
    # PRINT TO PREVIEW
    # __FIELDS__ = df.columns
    # print(df.sample(5))
    # print("Data describe: ", df.describe())
    # print("Data size: ", df.count())
    # print(__FIELDS__)

    #========================Counting=========================#
    # Get only 'tags' column for counting
    series = df.tags
    __TAG_COUNT_DICT__, __SINGLE_TAG_COUNT_DICT__ = count_by_tag(__TAGSET__, series)

    d = {}
    for tag in __TAGSET__: 
        d[tag] = (__TAG_COUNT_DICT__[tag], __SINGLE_TAG_COUNT_DICT__[tag])

    # Combine the two dicts to one dataframe
    df_res = pd.DataFrame.from_dict(d, orient='index')
    df_res = df_res.reset_index()
    df_res.columns=['TagId', 'N Document', 'N SglTag Document']

    # Write to file
    # w_name = '/content/summary/' + file.split('/')[-1].split('.')[0]+'_summary.csv'
    # df_res.to_csv(w_name, header=True)


    #========================Extracting=========================#

    # df1 = df[['id', 'tags']]
    # __SINGLE_TAG__.append(get_sgl_tag_doc(df1))
    # ids = [r.id for r in __SINGLE_TAG__]
    # tags = [r.tags for r in __SINGLE_TAG__]
    # pd.DataFrame(data=[ids, tags]).transpose()



                  TagId  N Document  N SglTag Document
0   F-Goodingeneral      9568        6767             
1   N-SpoiledMilk        0           0                
2   O-Expirationdate     328         90               
3   F-Innerheat          6446        1033             
4   E-Loyalconsumers     1040        369              
..               ...      ...        ...              
59  MKT-Gift             3307        2773             
60  PD-Quality           960         262              
61  MKT-CustomerService  164         37               
62  F-Diarrhea           372         208              
63  N-Milkbubble         374         289              

[64 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                  TagId  N Document  N SglTag Document
0   F-Goodingeneral      12174       8057             
1   N-SpoiledMilk        0           0                
2   O-Expirationdate     672         206              
3   F-Innerheat          9527        1855             
4   E-Loyalconsumers     526         178              
..               ...     ...         ...              
59  MKT-Gift             7933        5946             
60  PD-Quality           9190        3384             
61  MKT-CustomerService  205         66               
62  F-Diarrhea           207         91               
63  N-Milkbubble         53          33               

[64 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                  TagId  N Document  N SglTag Document
0   F-Goodingeneral      2361        1788             
1   N-SpoiledMilk        0           0                
2   O-Expirationdate     594         168              
3   F-Innerheat          321         117              
4   E-Loyalconsumers     145         63               
..               ...     ...         ..               
59  MKT-Gift             1869        1523             
60  PD-Quality           976         297              
61  MKT-CustomerService  100         17               
62  F-Diarrhea           27          19               
63  N-Milkbubble         9           3                

[64 rows x 3 columns]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                  TagId  N Document  N SglTag Document
0   F-Goodingeneral      1548        1130             
1   N-SpoiledMilk        0           0                
2   O-Expirationdate     536         146              
3   F-Innerheat          277         104              
4   E-Loyalconsumers     128         58               
..               ...     ...         ..               
59  MKT-Gift             1645        1346             
60  PD-Quality           794         235              
61  MKT-CustomerService  75          17               
62  F-Diarrhea           12          6                
63  N-Milkbubble         9           3                

[64 rows x 3 columns]


Note:
- Tag -1
- duplicated tag

In [ ]:
tagset_stored = zip(range(0, len(__TAGSET__)), __TAGSET__)

In [ ]:
tagset_stored